In [2]:
from selenium import webdriver
from bs4 import BeautifulSoup
import pandas as pd
import re
import requests
import numpy as np
from multiprocessing import  Pool
import time

In [3]:
## We first downloaded
dohData2 = pd.read_csv('Export_doh_bom.csv')

In [15]:
dohTest = dohData2.copy()

In [16]:
num_partitions = 2 #number of partitions to split dataframe
num_cores = 4 #number of cores on your machine
errors = []
emails = []
finalData = pd.DataFrame()

In [17]:
def parallelize_dataframe(df, func):
    df_split = np.array_split(df, num_partitions)
    pool = Pool(num_cores)
    df = pd.concat(pool.map(func, df_split))
    pool.close()
    pool.join()
    return df

In [18]:
def getEmails(dohTest):
    chrome_options = webdriver.ChromeOptions()
    chrome_options.add_argument('--no-sandbox')
    chrome_options.add_argument('--headless')
    chrome = webdriver.Chrome("/usr/lib/chromium-browser/chromedriver", options=chrome_options)
    for i in dohTest.iterrows():        
        try:
            ## Get target website as following ##
            chrome.get(i[1]['Urls'])

            ## Navigate to Practitioner Profile Tab ##
            chrome.find_element_by_link_text("Practitioner Profile").click()

            ## Get the page source from there to find email address ##
            content = chrome.page_source
            soup = BeautifulSoup(content)
            
            ## Check only General Information tab to obtain email address #
            ## Email address is stored within strong tags, so we find strong tag
            try:                
                emails.append(soup.find_all("div", {"id":"General"})[0].find_all("strong")[0].get_text())
            except:                
                emails.append("N/A")
            
        except:
            errors.append(i)
            emails.append("N/A")  
           
    dohTest['Email'] = emails
    return dohTest

In [ ]:
start_time = time.time()
finalData = parallelize_dataframe(dohTest, getEmails)
print("--- %s seconds ---" % (time.time() - start_time))

In [20]:
finalData.to_csv('finalBoard.csv')

In [14]:
finalData

,Unnamed: 0,License,Name,Profession,City,License Status,Urls,Email
0,0,OS16277,"Aaronson, Hadassah E.",Osteopathic Physician,HOLLYWOOD,CLEAR/ACTIVE,https://appsmqa.doh.state.fl.us/MQASearchServi...,doctoraaronson@gmail.com
1,1,OS9611,"ABAD, ENZO LUIS",Osteopathic Physician,HIALEAH,CLEAR/ACTIVE,https://appsmqa.doh.state.fl.us/MQASearchServi...,N/A
2,2,OS14999,"Abbate, Robert",Osteopathic Physician,DALLAS,CLEAR/ACTIVE,https://appsmqa.doh.state.fl.us/MQASearchServi...,doctorabbate@gmail.com
3,3,OS13361,"ABBOTT, GREGORY ALAN",Osteopathic Physician,WILLIAMSBURG,CLEAR/ACTIVE,https://appsmqa.doh.state.fl.us/MQASearchServi...,phsabbott@yahoo.com
4,4,OS8233,"ABDALLA, EMIL VINCENT",Osteopathic Physician,PEMBROKE PINES,CLEAR/ACTIVE,https://appsmqa.doh.state.fl.us/MQASearchServi...,eabdalla@femwell.com
5,5,OS8138,"ABDALLAH, MOHAMMED",Osteopathic Physician,DEERFIELD BEACH,CLEAR/ACTIVE,https://appsmqa.doh.state.fl.us/MQASearchServi...,drvasc@me.com
6,6,OS12121,"ABDALLAH, NADER ABDALLAH",Osteopathic Physician,WELLINGTON,CLEAR/ACTIVE,https://appsmqa.doh.state.fl.us/MQASearchServi...,drnaderabdallah@gmail.com
7,7,OS14131,"ABDALLAH, ODAY KARIM",Osteopathic Physician,LAKELAND,CLEAR/ACTIVE,https://appsmqa.doh.state.fl.us/MQASearchServi...,oabdallah@watsonclinic.com
8,8,OS15768,"Abdel Mageed, Engy",Osteopathic Physician,BRADENTON,CLEAR/ACTIVE,https://appsmqa.doh.state.fl.us/MQASearchServi...,emageed90@gmail.com
9,9,OS15678,"Abdoly, Amir",Osteopathic Physician,FT LAUDERDALE,CLEAR/ACTIVE,https://appsmqa.doh.state.fl.us/MQASearchServi...,aabdoly@browardhealth.org
